In [1]:
import synonyms

Building prefix dict from /data2/cuimengjun/anaconda3/envs/d2l/lib/python3.8/site-packages/synonyms/data/vocab.txt ...
Loading model from cache /tmp/jieba.u1ee567e3d900769013ee0717e970f060.cache



 Synonyms: v3.18.0, Project home: https://github.com/chatopera/Synonyms/

 Project Sponsored by Chatopera

  deliver your chatbots with Chatopera Cloud Services --> https://bot.chatopera.com

>> Synonyms load wordseg dict [/data2/cuimengjun/anaconda3/envs/d2l/lib/python3.8/site-packages/synonyms/data/vocab.txt] ... 


Loading model cost 1.469 seconds.
Prefix dict has been built successfully.


>> Synonyms on loading stopwords [/data2/cuimengjun/anaconda3/envs/d2l/lib/python3.8/site-packages/synonyms/data/stopwords.txt] ...
>> Synonyms on loading vectors [/data2/cuimengjun/anaconda3/envs/d2l/lib/python3.8/site-packages/synonyms/data/words.vector.gz] ...


In [2]:
synonyms.nearby("爸爸")

(['爸爸', '妈妈', '爷爷', '奶奶', '老婆', '姐姐', '爸妈', '老公', '老爸', '外婆'],
 [1.0,
  0.9197904,
  0.8369896,
  0.83030784,
  0.80404544,
  0.79813045,
  0.7979228,
  0.7714372,
  0.77116084,
  0.7584483])

In [3]:
stop_words = {word.strip() for word in open('data/baidu_stopwords.txt', 'r', encoding='utf-8').readlines()}

In [4]:
# 同义词替换(SR): 从句子中随机选择n个不是停用词的词 用随机选择的同义词之一替换这些单词中的每一个
import random

def get_synonyms(word):
    sys = set(synonyms.nearby(word)[0])
    if word in sys:
        sys.remove(word)
        
    return list(sys)

def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced > n:
            break
            
    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')
    
    return new_words

In [5]:
# 随机删除(RD): 以概率p随机删除句子中的每个单词

def random_deletion(words, p):
    if len(words) == 1:
        return words
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)
            
    if len(new_words) == 0:
        random_int = random.randint(0, len(words) - 1)
        return [words[random_int]]
    
    return new_words

In [6]:
# 随机交换(RS): 随机选择句子中的两个单词并交换他们的位置 做n次
def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words) - 1)
    random_idx_2 = random_idx_1
    
    cnt = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words) - 1)
        cnt += 1
        if cnt > 3:
            return new_words
        
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    
    return new_words

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    
    return new_words

In [7]:
# 随即插入(RI): 在句子中随机找到一个词, 并找出其同义词, 且该词不是停用词, 将该同义词插入句子的随机位置 做n次
def add_word(new_words):
    synonyms = []
    cnt = 0
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words) - 1)]
        synonyms = get_synonyms(random_word)
        cnt += 1
        if cnt >= 10:
            return
    
def random_insert(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
        
    return new_words

In [8]:
def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
    words = synonyms.seg(sentence)[0]
    num_words = len(words)
    augmented_sentence = []
    num_new_per_technique = int(num_aug / 4) + 1
    
    n_sr = max(1, (int)(alpha_sr * num_words))
    n_ri = max(1, int(alpha_ri*num_words))
    n_rs = max(1, int(alpha_rs*num_words))
    
    # sr
    for _ in range(num_new_per_technique):
        a_words = synonym_replacement(words, n_sr)
        augmented_sentence.append(' '.join(a_words))
    # ri
    for _ in range(num_new_per_technique):
        a_words = random_insert(words, n_ri)
        augmented_sentence.append(' '.join(a_words))
    # rs
    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentence.append(' '.join(a_words))
    # rd
    for _ in range(num_new_per_technique):
        a_words = random_deletion(words, p_rd)
        augmented_sentence.append(' '. join(a_words))
        
    random.shuffle(augmented_sentence)
    if num_aug >= 1:
        augmented_sentence = augmented_sentence[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentence)
        augmented_sentence = [s for s in augmented_sentence if random.uniform(0, 1) < keep_prob]
    
    return augmented_sentence

In [9]:
words = synonyms.seg('目前华为部分型号的手机产品出现货少的现象')
print(words)

(['目前', '华为', '部分', '型号', '的', '手机', '产品', '出现', '货', '少', '的', '现象'], ['t', 'nr', 'n', 'n', 'uj', 'n', 'n', 'v', 'n', 'a', 'uj', 'n'])


In [10]:
eda('9月15日以来，台积电、高通、三星等华为的重要合作伙伴，只要没有美国的相关许可证，都无法供应芯片给华为，而中芯国际等国产芯片企业，也因采用美国技术，而无法供货给华为。目前华为部分型号的手机产品出现货少的现象，若该形势持续下去，华为手机业务将遭受重创。')

['15 日 以来 ， 台积电 、 高通 、 三星 华为 的 重要 合作伙伴 ， 没有 美国 的 相关 ， 都 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 少 的 现象 ， 若 该 形势 持续 下去 ， 华为 手机 业务 将 遭受 重创 。',
 '9 月 三星 日 以来 ， 台积电 、 高通 、 15 等 华为 的 重要 合作伙伴 ， ， 没有 给 的 相关 许可证 ， 都 无法 供应 芯片 美国 华为 企业 而 中芯国际 等 国产 芯片 只要 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 业务 的 该 ， 若 现象 形势 华为 下去 ， 持续 手机 少 将 遭受 重创 。',
 '9 月 15 日 以来 ， Tektronix 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 全都 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 化工企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 机型 的 笔记型电脑 产品 出现 货品 贵 的 现象 ， 若 该 形势 持续 下去 ， 华为 笔记型电脑 业务 将 遭受 受创 。',
 '9 月 15 日 以来 ， 台积电 、 高通 、 三星 等 华为 的 重要 SAP ， 只要 没有 英国 的 相关 许可证 ， 即便 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 导入 英国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 多一点 的 弊病 ， 若 该 宏观经济形势 持续 下去 ， 华为 手机 业务 将 遭受 重挫 。',
 '9 月 15 日 以来 ， 台积电 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 少 的 现象 ， 若 

#### 闭包数据增强
数据集中每条数据有两个句子\
a和b相似 a和c相似\
则b和c相似\
a和d不相似 0\
则b和d不相似

In [11]:
import pandas as pd
from tqdm.notebook import tqdm
import json

def parse_data(path, data_type='train'):
    sentence_a, sentence_b, labels = [], [], []
    with open(path, 'r', encoding='utf-8') as f:
        for line in tqdm(f.readlines(), desc=f'Reading {data_type} data'):
            line = json.loads(line)
            sentence_a.append(line['sentence1'])
            sentence_b.append(line['sentence2'])
            if data_type != 'test':
                labels.append(int(line['label']))
            else:
                labels.append(0)
    df = pd.DataFrame(zip(sentence_a, sentence_b, labels), columns=['text_a', 'text_b', 'labels'])
    
    return df

In [12]:
train_df = parse_data('data/train.json', data_type='train')

Reading train data:   0%|          | 0/34334 [00:00<?, ?it/s]

In [13]:
train_df.head(3)

,text_a,text_b,labels
0,蚂蚁借呗等额还款可以换成先息后本吗,借呗有先息到期还本吗,0
1,蚂蚁花呗说我违约一次,蚂蚁花呗违约行为是什么,0
2,帮我看一下本月花呗账单有没有结清,下月花呗账单,0


In [14]:
for data in train_df.groupby(by=['text_a']):
    if len(data[1]) == 3:
        print(data[0])
        print(type(data[1]))
        print(data[1])
        break

为何我的花呗不能用
<class 'pandas.core.frame.DataFrame'>
          text_a       text_b  labels
13425  为何我的花呗不能用     为什么花呗用不了       1
14005  为何我的花呗不能用    花呗不能正常使用了       1
15175  为何我的花呗不能用  我现在是不是用不了花呗       0


In [15]:
from collections import defaultdict

def aug_group_by_a(df):
    aug_data = defaultdict(list)
    # 以text_a句子为a
    for g, data in df.groupby(by=['text_a']):
        if len(data) < 2:
            continue
        for i in range(len(data)):
            for j in range(i + 1, len(data)):
                # 取出b的值 a, b的label
                row_i_text = data.iloc[i, 1]
                row_i_label = data.iloc[i, 2]
                # 取出c的值 a, c的label
                row_j_text = data.iloc[j, 1]
                row_j_label = data.iloc[j, 2]
                
                if row_i_label == row_j_label == 0:
                    continue
                aug_label = 1 if row_i_label == row_j_label == 1 else 0
                
                aug_data['text_a'].append(row_i_text)
                aug_data['text_b'].append(row_j_text)
                aug_data['labels'].append(aug_label)
    
    return pd.DataFrame(aug_data)

In [16]:
aug_train_a = aug_group_by_a(train_df)
print(len(aug_train_a))

228


In [17]:
from extra_file.bucket_sampler import SortedSampler, BucketBatchSampler
from extra_file.EMA import *

In [55]:
import random
import numpy as np
import torch

config = {
        'train_file_path': 'data/train.json',
        'dev_file_path': 'data/dev.json',
        'test_file_path': 'data/test.json',
        'output_path': '.',
        'model_path': '../NLP_Project/dataset/BERT_model/',
        'batch_size': 16,
        'num_epochs': 2,
        'max_seq_len': 64,
        'learning_rate': 2e-5,
        'weight_decay': 0.01,
        'use_bucket': True,
        'bucket_multiplier': 200,
        'unsup_data_ratio': 1.5,
        'uda_softmax_temp': 0.4,
        'uda_confidence_threshold': 0.8,
        'device': 'cuda',
        'n_gpus': 0,
        'logging_step': 400,
        'ema_start_step': 500,
        'ema_start': False,
        'seed': 2021
}

if not torch.cuda.is_available():
    config['device'] = 'cpu'
else:
    config['n_gpus'] = torch.cuda.device_count()
    config['batch_size'] *= config['n_gpus']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2021

In [19]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(config['model_path'])

In [20]:
def build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
    inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens=True,
                          return_token_type_ids=True, return_attention_mask=True)
    
    inputs['input_ids'].append(inputs_dict['input_ids'])
    inputs['token_type_ids'].append(inputs_dict['token_type_ids'])
    inputs['attention_mask'].append(inputs_dict['attention_mask'])
    inputs['labels'].append(label)

In [21]:
# 对偶数据增强
def build_unsup_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
    lr_inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens=True,
                          return_token_type_ids=True, return_attention_mask=True)
                     
    rl_inputs_dict = tokenizer.encode_plus(sentence_b, sentence_a, add_special_tokens=True,
                          return_token_type_ids=True, return_attention_mask=True)
    
    inputs['input_ids'].append((lr_inputs_dict['input_ids'], rl_inputs_dict['input_ids']))
    inputs['token_type_ids'].append((lr_inputs_dict['token_type_ids'],rl_inputs_dict['token_type_ids']))
    inputs['attention_mask'].append((lr_inputs_dict['attention_mask'],rl_inputs_dict['attention_mask']))
    inputs['labels'].append(label)

In [22]:
def read_data(config, tokenizer):
    train_df = parse_data(config['train_file_path'], data_type='train')
    dev_df = parse_data(config['dev_file_path'], data_type='dev')
    test_df = parse_data(config['test_file_path'], data_type='test')
    
    data_df = {'train': train_df, 'dev': dev_df, 'test': test_df}
    processed_data = {}
    unsup_data = defaultdict(list)
    
    for data_type, df in data_df.items():
        inputs = defaultdict(list)
        if data_type == 'train':
            reversed_inputs = defaultdict(list)
        
        for i, row in tqdm(df.iterrows(), desc=f'Processing {data_type} data', total=len(df)):
            label = 0 if data_type == 'test' else row[2]
            sentence_a, sentence_b = row[0], row[1]
            build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer)
            
            if data_type.startswith("test"):
                build_bert_inputs(inputs, label, sentence_b, sentence_a, tokenizer)
            
            build_unsup_bert_inputs(unsup_data, label, sentence_a, sentence_b, tokenizer)
        processed_data[data_type] = inputs
        
    processed_data['unsup_data'] = unsup_data
    
    return processed_data

In [23]:
data = read_data(config, tokenizer)

Reading train data:   0%|          | 0/34334 [00:00<?, ?it/s]

Reading dev data:   0%|          | 0/4316 [00:00<?, ?it/s]

Reading test data:   0%|          | 0/3861 [00:00<?, ?it/s]

Processing train data:   0%|          | 0/34334 [00:00<?, ?it/s]

Processing dev data:   0%|          | 0/4316 [00:00<?, ?it/s]

Processing test data:   0%|          | 0/3861 [00:00<?, ?it/s]

In [24]:
from torch.utils.data import Dataset

class AFQMCDataset(Dataset):
    def __init__(self, data_dict):
        super(AFQMCDataset, self).__init__()
        self.data_dict = data_dict
        
    def __getitem__(self, idx):
        data = (self.data_dict['input_ids'][idx], self.data_dict['token_type_ids'][idx],
                self.data_dict['attention_mask'][idx], self.data_dict['labels'][idx])
        return data
    
    def __len__(self):
        return len(self.data_dict['input_ids'])

In [25]:
class Collator:
    def __init__(self, max_seq_len, tokenizer):
        self.max_seq_len = max_seq_len
        self.tokenizer = tokenizer
        
    def pad_and_truncate(self, input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len):
        input_ids = torch.zeros((len(input_ids_list), max_seq_len), dtype=torch.long)
        token_type_ids = torch.zeros_like(input_ids)
        attention_mask = torch.zeros_like(input_ids)
        
        for i in range(len(input_ids_list)):
            seq_len = len(input_ids_list[i])
            if seq_len <= max_seq_len:
                input_ids[i, :seq_len] = torch.tensor(input_ids_list[i], dtype=torch.long)
                token_type_ids[i, :seq_len] = torch.tensor(token_type_ids_list[i], dtype=torch.long)
                attention_mask[i, :seq_len] = torch.tensor(attention_mask_list[i], dtype=torch.long)
            else:
                input_ids[i] = torch.tensor(input_ids_list[i][:max_seq_len-1] + [self.tokenizer.sep_token_id], dtype=torch.long)
                token_type_ids[i] = torch.tensor(token_type_ids_list[i][:max_seq_len], dtype=torch.long)
                attention_mask[i] = torch.tensor(attention_mask_list[i][:max_seq_len], dtype=torch.long)
        
        labels = torch.tensor(labels_list, dtype=torch.long)
        
        return input_ids, token_type_ids, attention_mask, labels
        
    def __call__(self, examples):
        input_ids_list, token_type_ids_list, attention_mask_list, labels_list = list(zip(*examples))
        cur_max_seq_len = max(len(input_ids) for input_ids in input_ids_list)
        max_seq_len = min(cur_max_seq_len, self.max_seq_len)
        
        input_ids, token_type_ids, attention_mask, labels = self.pad_and_truncate(input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len)
        data_dict = {
            'input_ids': input_ids,
            'token_type_ids': token_type_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }
        return data_dict

In [26]:
collate_fn = Collator(config['max_seq_len'], tokenizer)

In [27]:
# UDA
class UnsupAFQMCDataset(Dataset):
    def __init__(self, data_dict):
        super(UnsupAFQMCDataset, self).__init__()
        self.data_dict = data_dict
        
    def __getitem__(self, idx):
        input_ids = self.data_dict['input_ids'][idx]
        token_type_ids = self.data_dict['token_type_ids'][idx]
        attention_mask = self.data_dict['attention_mask'][idx]
        labels = self.data_dict['labels'][idx]
        
        return (input_ids[0], token_type_ids[0], attention_mask[0],
                 input_ids[1], token_type_ids[1], attention_mask[1],
                 labels)
    
    def __len__(self):
        return len(self.data_dict['input_ids'])

In [28]:
class UnsupCollator(Collator):
    def __init__(self, max_seq_len, tokenizer):
        self.max_seq_len = max_seq_len
        self.tokenizer = tokenizer
        
    def __call__(self, examples):
        (ab_input_ids_list, ab_token_type_ids_list, ab_attention_mask_list,
         ba_input_ids_list, ba_token_type_ids_list, ba_attention_mask_list, labels_list) = list(zip(*examples))
        
        cur_max_seq_len = max(len(input_ids) for input_ids in ab_input_ids_list)
        max_seq_len = min(cur_max_seq_len, self.max_seq_len)
        
        # 分批整ab, ba
        ab_input_ids, ab_token_type_ids, ab_attention_mask, labels = self.pad_and_truncate(ab_input_ids_list, ab_token_type_ids_list, ab_attention_mask_list, labels_list, max_seq_len)
        ba_input_ids, ba_token_type_ids, ba_attention_mask, labels = self.pad_and_truncate(ba_input_ids_list, ba_token_type_ids_list, ba_attention_mask_list, labels_list, max_seq_len)

        data_dict = {
            'ab_input_ids': ab_input_ids,
            'ab_token_type_ids': ab_token_type_ids,
            'ab_attention_mask': ab_attention_mask,
            'ba_input_ids': ba_input_ids,
            'ba_token_type_ids': ba_token_type_ids,
            'ba_attention_mask': ba_attention_mask,
            'labels': labels

        }
        return data_dict

In [43]:
from torch.utils.data import DataLoader, RandomSampler
from extra_file.bucket_sampler import BucketBatchSampler

def build_dataloader(config, data, tokenizer):
    train_dataset = AFQMCDataset(data['train'])
    dev_dataset = AFQMCDataset(data['dev'])
    test_dataset = AFQMCDataset(data['test'])
    unsup_dataset = UnsupAFQMCDataset(data['unsup_data'])
    
    collate_fn = Collator(config['max_seq_len'], tokenizer)
    unsup_collate_fn = UnsupCollator(config['max_seq_len'], tokenizer)
    
    if config['use_bucket']:
        # 监督数据
        train_sampler = RandomSampler(train_dataset)
        bucket_sampler = BucketBatchSampler(train_sampler, batch_size=config['batch_size'],
                                            drop_last=False, sort_key=lambda x: len(train_dataset[x][0]),
                                            bucket_size_multiplier=config['bucket_multiplier'])
        train_dataloader = DataLoader(dataset=train_dataset, batch_sampler=bucket_sampler, num_workers=4, collate_fn=collate_fn)
        
        # 无监督数据
        unsup_sampler = RandomSampler(unsup_dataset)
        unsup_bucket_sampler = BucketBatchSampler(unsup_sampler, batch_size=int(config['batch_size'] * config['unsup_data_ratio']),
                                                  drop_last=False, sort_key=lambda x: len(unsup_dataset[x][0]),
                                                  bucket_size_multiplier=config['bucket_multiplier'])
        unsup_dataloader = DataLoader(dataset=unsup_dataset, batch_sampler=unsup_bucket_sampler, num_workers=4, collate_fn=unsup_collate_fn)
    
    else:
        # 监督数据
        train_dataloader = DataLoader(dataset=train_dataset, batch_size=config['batch_size'],
                                     shuffle=True, num_workers=4, collate_fn=collate_fn)

        # 无监督数据
        unsup_dataloader = DataLoader(dataset=unsup_dataset, batch_size=int(config['batch_size']*config['unsup_data_ratio']),
                                      shuffle=True, num_workers=4, collate_fn=unsup_collate_fn)
    
    dev_dataloader = DataLoader(dataset=dev_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4, collate_fn=collate_fn)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4, collate_fn=collate_fn)

    return unsup_dataloader, train_dataloader, dev_dataloader, test_dataloader

In [44]:
unsup_dataloader, train_dataloader, dev_dataloader, test_dataloader = build_dataloader(config, data, tokenizer)

In [45]:
for i in train_dataloader:
    print(i)
    break

{'input_ids': tensor([[ 101, 2769, 7566,  ..., 1446,  102,    0],
        [ 101, 6010, 6009,  ..., 3299,  102,    0],
        [ 101, 2769, 4638,  ..., 4638,  102,    0],
        ...,
        [ 101, 2769, 1762,  ..., 1446, 1568,  102],
        [ 101, 5709, 1446,  ..., 6589, 1408,  102],
        [ 101, 2769, 4638,  ..., 1184, 6820,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 0],
        [0, 0, 0,  ..., 1, 1, 0],
        [0, 0, 0,  ..., 1, 1, 0],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0

In [36]:
from sklearn import metrics

def evaluation(config, model, val_dataloader):
    model.eval()
    preds = []
    labels = []
    val_loss = 0.
    val_iterator = tqdm(val_dataloader, desc='Evaluation', total=len(val_dataloader))

    with torch.no_grad():
        for batch in val_iterator:
            labels.append(batch['labels'])
            batch_cuda = {item: value.to(config['device']) for item, value in list(batch.items())}
            batch_cuda['mode'] = 'val'
            loss, logits = model(**batch_cuda)[:2]

            if config['n_gpus'] > 1:
                loss = loss.mean()

            val_loss += loss.item()
            preds.append(logits.argmax(dim=-1).detach().cpu())

    avg_val_loss = val_loss / len(val_dataloader)
    labels = torch.cat(labels, dim=0).numpy()
    preds = torch.cat(preds, dim=0).numpy()
    f1 = metrics.f1_score(labels, preds)
    acc = metrics.accuracy_score(labels, preds)
    return avg_val_loss, f1, acc

In [48]:
from transformers import BertForSequenceClassification
import torch.nn as nn

class BertForAFQMC(BertForSequenceClassification):
    def forward(self,
                input_ids,
                token_type_ids,
                attention_mask,
                labels=None,
                mode='train'):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_hidden_states=True
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        
        outputs = (logits, )

        if mode == 'val':
            loss_fct = nn.CrossEntropyLoss()

            loss = loss_fct(logits, labels.view(-1))
                
            outputs = (loss,) + outputs

        return outputs

In [38]:
# Return grad_data: 需要计算梯度 需要进行反向传播的数据
# Return no_grad_data: 不需要计算梯度 不需要进行反向传播的数据
def get_data(sup_batch, unsup_batch, config):
    grad_data, no_grad_data = {}, {}
    # sup_batch [bs, seq_len]  unsup_batch [bs, seq_len]
    sup_max_len = sup_batch['input_ids'].size(1)
    unsup_max_len = unsup_batch['ba_input_ids'].size(1)
    cur_max_len = max(sup_max_len, unsup_max_len)
    
    for item, sup_value in sup_batch.items():
        if item == 'labels':
            grad_data[item] = sup_value.to(config['device'])
            continue
        
        ba_unsup_value, ab_unsup_value = unsup_batch[f'ba_{item}'], unsup_batch[f'ab_{item}']
        # 谁短补水
        if sup_max_len == cur_max_len:
            padding_value = torch.zeros((ba_unsup_value.size(0), cur_max_len - unsup_max_len),
                                        dtype=ba_unsup_value.dtype)
            ba_unsup_value = torch.cat([ba_unsup_value, padding_value], dim=-1)
        else:
            padding_value = torch.zeros((sup_value.size(0), cur_max_len - sup_max_len),
                                        dtype=sup_value.dtype)
            sup_value = torch.cat([sup_value, padding_value], dim=-1)
        # sup_batch ba cat
        grad_value = torch.cat([sup_value, ba_unsup_value], dim=0)
        grad_data[item] = grad_value.to(config['device'])
        no_grad_data[item] = ab_unsup_value.to(config['device'])
        
    return grad_data, no_grad_data

In [40]:
logits = torch.randn(2,3)
print(logits)
t_softmax = torch.softmax(logits, dim=1)
print(t_softmax)
t_sharpen = torch.softmax(logits/0.4, dim=1)
print(t_sharpen)

tensor([[ 0.3431,  0.2891,  0.0157],
        [-0.8506, -0.2726,  0.2910]])
tensor([[0.3748, 0.3551, 0.2702],
        [0.1691, 0.3014, 0.5295]])
tensor([[0.4320, 0.3774, 0.1906],
        [0.0442, 0.1877, 0.7680]])


In [39]:
# 无监督数据 ab 只需要正向传播
def forward_no_grad(no_grad_data, config, model):
    with torch.no_grad():
        no_grad_logits = model(**no_grad_data)[0]
        # sharpen
        no_grad_probs = torch.softmax(no_grad_logits / config['uda_softmax_temp'], dim=-1)
        largest_probs, _ = no_grad_probs.max(dim=-1)
        unsup_loss_mask = largest_probs.gt(config['uda_confidence_threshold']).float()
        # unsup_loss_mask tensor9[True, False, True, False]
    return unsup_loss_mask, no_grad_probs

In [41]:
def get_tsa_threshold(total_steps, global_steps):
    return np.exp((global_steps / total_steps - 1) * 5) / 2 + 0.5

In [52]:
def forward_with_grad(unsup_loss_mask, unsup_probs, config, cur_bs,
                      model, grad_data, total_steps, global_steps):
    # 得到eta 随着训练进行阈值逐渐变大最后达到1 将所有的监督数据都使用了
    tsa_threshold = get_tsa_threshold(total_steps, global_steps)
    logits = model(**grad_data)[0]
    
    # 有监督损失部分
    # cur_bs 无监督ba的batch_size
    # 前面是train的sup_data 后面是unsup_data
    sup_logits, unsup_logits = logits.split([logits.size(0) - cur_bs, cur_bs])
    sup_labels = grad_data['labels'][:logits.size(0) - cur_bs]
    per_example_loss = nn.CrossEntropyLoss(reduction='none')(sup_logits, sup_labels)
    # 拿出正确标签对应的概率
    correct_label_probs = torch.softmax(sup_logits, dim=-1).gather(dim=-1, index=sup_labels.view(-1, 1))
    # 监督数据中过于自信的不要留下小于tsa_threshold的计算损失
    sup_loss_mask = correct_label_probs.le(tsa_threshold).squeeze().float()
    # 应用mask掩盖有监督数据中过于自信的样本
    per_example_loss *= sup_loss_mask
    sup_loss = per_example_loss.sum() / max(sup_loss_mask.sum(), 1) # 有效个数
    
    # 无监督损失部分
    unsup_log_probs = torch.log_softmax(unsup_logits, dim=-1)
    per_example_kl_loss = nn.KLDivLoss(reduction='none')(unsup_log_probs, unsup_probs).sum(dim=-1)
    # 应用mask掩盖无监督数据中不自信的样本
    per_example_kl_loss *= unsup_loss_mask
    unsup_loss = per_example_kl_loss.sum() / max(unsup_loss_mask.sum(), 1)
    
    # 可以后续加权两种损失
    loss = unsup_loss + sup_loss
    
    if config['n_gpus'] > 1:
        loss = loss.mean()
        sup_loss = sup_loss.mean()
        unsup_loss = unsup_loss.mean()
        
    return loss, tsa_threshold, unsup_loss, sup_loss

In [53]:
from torch.optim import AdamW
from tqdm.notebook import trange
import os

def train(config, train_dataloader, dev_data_loader, unsup_dataloader=None):
    model = BertForAFQMC.from_pretrained(config['model_path'])
    optimizer = AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    model.to(config['device'])
    # unsup_dataloader: train + dev + test 最大
    total_steps = len(unsup_dataloader) * config['num_epochs']
    epoch_iterator = trange(config['num_epochs'])
    global_steps = 0
    train_loss = 0.
    logging_loss = 0.
    best_acc = 0.
    best_model_path = ''
    
    if config['n_gpus'] > 1:
        model = nn.DataParallel(model)
        
    train_iterator = iter(train_dataloader)
    for _ in epoch_iterator:
        unsup_iterator = tqdm(unsup_dataloader, desc='Training', total=len(unsup_dataloader))
        model.train()
        for unsup_batch in unsup_iterator:
            cur_bs = unsup_batch['ab_input_ids'].size(0)
            try:
                sup_batch = next(train_iterator)
            except StopIteration:
                train_iterator = iter(train_dataloader)
                sup_batch = next(train_iterator)
            
            grad_data, no_grad_data = get_data(sup_batch, unsup_batch, config)
            
            unsup_loss_mask, unsup_probs = forward_no_grad(no_grad_data, config, model)
            loss, tsa_threshold, unsup_loss, sup_loss = forward_with_grad(
                unsup_loss_mask, unsup_probs, config, cur_bs, model, grad_data, total_steps, global_steps)
            
            model.zero_grad()
            loss.backward()
            
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            if config['ema_start']:
                ema.update()
            
            train_loss += loss.item()
            global_steps += 1
            
            unsup_iterator.set_postfix_str(f'running training loss: {loss.item():.4f}')
            if global_steps % config['logging_step'] == 0:
                if global_steps >= config['ema_start_step'] and not config['ema_start']:
                    print('\n>>> EMA starting ...')
                    config['ema_start'] = True
                    ema = EMA(model.module if hasattr(model, 'module') else model, decay=0.999)

                print_train_loss = (train_loss - logging_loss) / config['logging_step']
                logging_loss = train_loss

                if config['ema_start']:
                    ema.apply_shadow()
                val_loss, f1, acc = evaluation(config, model, dev_dataloader)

                print_log = f'\n>>> training loss: {print_train_loss:.6f}, valid loss: {val_loss:.6f}, '

                if acc > best_acc:
                    model_save_path = os.path.join(config['output_path'],
                                                   f'checkpoint-{global_steps}-{acc:.6f}')
                    model_to_save = model.module if hasattr(model, 'module') else model
                    model_to_save.save_pretrained(model_save_path)
                    best_acc = acc
                    best_model_path = model_save_path

                print_log += f'valid f1: {f1:.6f}, valid acc: {acc:.6f}'

                print(print_log)
                model.train()
                if config['ema_start']:
                    ema.restore()

    return model, best_model_path

In [56]:
model, best_model_path = train(config, train_dataloader, dev_dataloader, unsup_dataloader)

Some weights of the model checkpoint at ../NLP_Project/dataset/BERT_model/ were not used when initializing BertForAFQMC: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForAFQMC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForAFQMC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForAFQMC were not initialized from the model checkpoint at ../NLP_Project/dataset/BERT_model/ and are newly i

  0%|          | 0/2 [00:00<?, ?it/s]

Training:   0%|          | 0/443 [00:00<?, ?it/s]

Evaluation:   0%|          | 0/68 [00:00<?, ?it/s]


>>> training loss: 1.041689, valid loss: 0.596706, valid f1: 0.473610, valid acc: 0.653383


Training:   0%|          | 0/443 [00:00<?, ?it/s]


>>> EMA starting ...


Evaluation:   0%|          | 0/68 [00:00<?, ?it/s]


>>> training loss: 0.992483, valid loss: 0.530302, valid f1: 0.478155, valid acc: 0.701112
